In [7]:
pip install yfinance



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.6/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import yfinance as yf
import pandas as pd
import ta

# Define tickers and time range
tickers = ['AAPL', 'TSLA', 'AMZN', 'MSFT', 'GOOGL', 'META', 'JNJ', 'JPM', 'NVDA', 'V']
start_date = '2019-01-01'
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# Download the data
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', auto_adjust=True)

# Prepare an empty list to collect processed DataFrames
all_stocks = []

# Loop through each ticker to create indicators and clean structure
for ticker in tickers:
    df = data[ticker].copy()
    df = df.dropna().reset_index()
    df['Ticker'] = ticker
    df['momentum_rsi'] = ta.momentum.RSIIndicator(df['Close']).rsi()
    df['trend_macd'] = ta.trend.MACD(df['Close']).macd()
    df['volatility_bbm'] = ta.volatility.BollingerBands(df['Close']).bollinger_mavg()
    df['trend_ema_fast'] = ta.trend.EMAIndicator(df['Close'], window=12).ema_indicator()
    all_stocks.append(df)

# Combine all stock data
historical_data = pd.concat(all_stocks)
historical_data = historical_data.dropna().reset_index(drop=True)

# Optional: Save to CSV
historical_data.to_csv("last_5yr_stock_indicators.csv", index=False)

# Preview
historical_data.head()


[*********************100%***********************]  10 of 10 completed


Price,Date,Open,High,Low,Close,Volume,Ticker,momentum_rsi,trend_macd,volatility_bbm,trend_ema_fast
0,2019-02-07,41.067104,41.433947,40.576396,40.719322,126966800,AAPL,66.468713,1.077211,38.044961,39.475621
1,2019-02-08,40.427456,40.826970,40.291094,40.767162,95280000,AAPL,66.690986,1.100447,38.251497,39.674320
2,2019-02-11,40.920285,40.958563,40.489670,40.532730,83973600,AAPL,64.436843,1.087409,38.464297,39.806383
3,2019-02-12,40.693010,40.908315,40.597316,40.882000,89134000,AAPL,66.266239,1.092665,38.721835,39.971863
4,2019-02-13,41.001617,41.262376,40.649948,40.712147,89960800,AAPL,64.527830,1.070781,38.934315,40.085752


In [11]:
historical_data.shape

(15880, 11)

In [12]:
# Load the dataset
df = pd.read_csv('last_5yr_stock_indicators.csv')


In [13]:
# Parse the 'Date' column into datetime format
df['Date'] = pd.to_datetime(df['Date'])


In [14]:
# Sort data by Ticker and Date
df = df.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

In [15]:
# Drop duplicate rows if any
df = df.drop_duplicates()

In [16]:
# Drop rows with missing values in key columns
df = df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'])


In [17]:

# Create a Daily Return column (percentage change of Close)
df['Daily_Return'] = df.groupby('Ticker')['Close'].pct_change() * 100


In [18]:
# Optional: round all float columns to 2 decimals for easier reading
float_cols = df.select_dtypes(include='float64').columns
df[float_cols] = df[float_cols].round(2)

In [19]:
# Show cleaned data info
print("✅ Cleaned dataset preview:")
print(df.head())

# Save cleaned dataset
df.to_csv('cleaned_stock_data.csv', index=False)

✅ Cleaned dataset preview:
        Date   Open   High    Low  Close     Volume Ticker  momentum_rsi  \
0 2019-02-07  41.07  41.43  40.58  40.72  126966800   AAPL         66.47   
1 2019-02-08  40.43  40.83  40.29  40.77   95280000   AAPL         66.69   
2 2019-02-11  40.92  40.96  40.49  40.53   83973600   AAPL         64.44   
3 2019-02-12  40.69  40.91  40.60  40.88   89134000   AAPL         66.27   
4 2019-02-13  41.00  41.26  40.65  40.71   89960800   AAPL         64.53   

   trend_macd  volatility_bbm  trend_ema_fast  Daily_Return  
0        1.08           38.04           39.48           NaN  
1        1.10           38.25           39.67          0.12  
2        1.09           38.46           39.81         -0.58  
3        1.09           38.72           39.97          0.86  
4        1.07           38.93           40.09         -0.42  


In [20]:
df.shape

(15880, 12)

In [21]:

df = pd.read_csv("cleaned_5yr_stock_data.csv")
print(df.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker',
       'momentum_rsi', 'trend_macd', 'volatility_bbm', 'trend_ema_fast',
       'Daily_Return'],
      dtype='object')


In [22]:
df = pd.read_csv("cleaned_5yr_stock_data.csv", parse_dates=["Date"])


In [24]:
df["Month"] = df["Date"].dt.to_period("M")

In [25]:
df = pd.read_csv("cleaned_5yr_stock_data.csv")
print(df.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker',
       'momentum_rsi', 'trend_macd', 'volatility_bbm', 'trend_ema_fast',
       'Daily_Return'],
      dtype='object')


In [28]:


df["Month"] = df["Date"].dt.to_period("M")
df["Month"] = df["Date"].dt.to_period("M")


In [29]:
df["Month"] = df["Date"].dt.to_period("M")


In [30]:
monthly_summary = df.groupby(["Ticker", "Month"]).agg({
    "Close": ["mean", "min", "max"],
    "Daily_Return": "std",
    "Volume": "sum",
    "momentum_rsi": "mean",
    "trend_macd": "mean",
    "volatility_bbm": "mean",
    "trend_ema_fast": "mean"
}).reset_index()
monthly_summary.columns = ['Ticker', 'Month', 'Mean_Close', 'Min_Close', 'Max_Close', 
                           'Volatility', 'Total_Volume', 'Avg_RSI', 'Avg_MACD', 'Avg_BB', 'Avg_EMA']


In [31]:
monthly_summary = monthly_summary.round(2)


In [32]:
df.to_csv("cleaned_5yr_stock_data_with_month.csv", index=False)
